# 03 · A/B Test (Simulated)
Random assignment, conversion uplift, frequentist tests + bootstrap CI.

In [5]:
import numpy as np, pandas as pd
from statsmodels.stats.proportion import proportions_ztest
from pathlib import Path
DATA = Path("../data")
rng = np.random.default_rng(42)

tx = pd.read_csv(DATA/"transactions.csv", parse_dates=["date"])

# A list of active users and randomly assign them to control or treatment
active_users = tx["user_id"].unique()
assign = pd.DataFrame({"user_id": active_users})
assign["variant"] = rng.choice(["control","treatment"], size=len(assign), p=[0.5,0.5])

In [6]:
# User-level conversion & Average Order Value (AOV) in recent window

# Define cutoff date (60 days within the latest transaction date)
cutoff = tx["date"].max() - pd.Timedelta(days=60)

# Filter transactions within the cutoff window
user_level = (tx[tx["date"]>=cutoff] 
              .groupby("user_id") # Group by user
              .agg(conversion=("cart_id", lambda s: int(s.nunique()>0)), # 1 if >=1 order, else 0
                   aov=("line_revenue", "mean")) # Average order value
              .reset_index()).fillna({"aov":0.0}) # Fill NaN AOV with 0.0

# Merge assignment with user-level metrics
df = assign.merge(user_level, on="user_id", how="left").fillna({"conversion":0, "aov":0.0})

In [7]:
df.head()

,user_id,variant,conversion,aov
0,3,treatment,1,10.49
1,1,control,1,562.79
2,2,treatment,1,141.95
3,4,treatment,1,280.00
4,8,control,1,9.85


In [ ]:
# Inject small uplift for demonstration
mask = df["variant"].eq("treatment")
uplift_rate = 0.10
df["conversion_sim"] = df["conversion"].astype(int)
flip = (df.loc[mask, "conversion_sim"]==0).values
df.loc[mask, "conversion_sim"] = np.where(flip, rng.binomial(1, uplift_rate, size=flip.sum()), 1)

# Two-proportion z-test
conv = df.groupby("variant")["conversion_sim"].agg(["sum","count"])
count = conv["sum"].values
nobs = conv["count"].values
stat, pval = proportions_ztest(count, nobs, alternative="larger")
print({
    "control_cr": float(count[0]/nobs[0]),
    "treatment_cr": float(count[1]/nobs[1]),
    "z": float(stat),
    "p_value": float(pval)
})

# Bootstrap 95% CI for uplift
def bootstrap_diff(a, b, B=5000, seed=0):
    rng = np.random.default_rng(seed)
    diffs = []
    for _ in range(B):
        aa = rng.choice(a, size=a.size, replace=True)
        bb = rng.choice(b, size=b.size, replace=True)
        diffs.append(bb.mean() - aa.mean())
    return np.percentile(diffs, [2.5, 97.5])

a = df.loc[df.variant=="control", "conversion_sim"].values
b = df.loc[df.variant=="treatment", "conversion_sim"].values
ci = bootstrap_diff(a, b)
print({"uplift_ci_95": [float(ci[0]), float(ci[1])]})